In [1]:
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk import FreqDist
from nltk.stem import PorterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
porter = PorterStemmer()
from sklearn.feature_extraction.text import TfidfVectorizer
import altair as alt
from vega_datasets import data

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/junggchangho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/junggchangho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
alt.renderers.enable('mimetype')

RendererRegistry.enable('mimetype')

In [3]:
humanist_vols = pd.read_csv("web_scraped_humanist_listserv.csv")
humanist_vols.head()

,dates,text
0,1987-1988,From: MCCARTY@UTOREPAS\nSubject: \nDate: 12 Ma...
1,1988-1989,From: Sebastian Rahtz \nSubject: C++ and Gnu o...
2,1989-1990,From: Willard McCarty \nSubject: Happy Birthda...
3,1990-1991,From: Elaine Brennan & Allen Renear \nSubject:...
4,1991-1992,From: Elaine Brennan & Allen Renear \nSubject:...


In [4]:
humanist_vols['year_start'] = humanist_vols['dates'].str.split('-').str[0]
humanist_vols['year_end'] = humanist_vols['dates'].str.split('-').str[1]
humanist_vols['volume_size'] = humanist_vols['text'].str.count('\n')
humanist_vols = humanist_vols.iloc[:21,:]
humanist_vols

,dates,text,year_start,year_end,volume_size
0,1987-1988,From: MCCARTY@UTOREPAS\nSubject: \nDate: 12 Ma...,1987,1988,50794
1,1988-1989,From: Sebastian Rahtz \nSubject: C++ and Gnu o...,1988,1989,36109
2,1989-1990,From: Willard McCarty \nSubject: Happy Birthda...,1989,1990,108695
3,1990-1991,From: Elaine Brennan & Allen Renear \nSubject:...,1990,1991,118929
4,1991-1992,From: Elaine Brennan & Allen Renear \nSubject:...,1991,1992,81193
5,1992-1993,From: Elaine M Brennan \nSubject: Humanist's B...,1992,1993,59957
6,1993-1994,From: 6500card%ucsbuxa@hub.ucsb.edu (Cheryl A....,1993,1994,71763
7,1994-1995,From: Andrew Burday \nSubject: Re: 7.0638 Qs: ...,1994,1995,46569
8,1995-1996,"From: ""Gregory Bloomquist"" \nSubject: Round Ta...",1995,1996,61076
9,1996-1997,From: Humanist \nSubject: Humanist begins its ...,1996,1997,52506


In [5]:
def stem_words(row):
    stemmed_words = ''
    for token in row.text.split(' '):
        stemmed_words += porter.stem(token) + ' '
    return stemmed_words
#subset_humanist_vols['stemmed_text'] = subset_humanist_vols.apply(stem_words, axis=1)
humanist_vols['stemmed_text'] = humanist_vols.apply(stem_words, axis=1)

In [6]:
documents = humanist_vols.text.tolist()
vectorizer = TfidfVectorizer(max_df=.7, min_df=1, stop_words=stop_words)

In [7]:
transformed_documents = vectorizer.fit_transform(documents)
transformed_documents_as_array = transformed_documents.toarray()
dates = humanist_vols.dates.tolist()
tfidf_results = []
for counter, doc in enumerate(transformed_documents_as_array):
    # construct a dataframe
    tf_idf_tuples = list(zip(vectorizer.get_feature_names(), doc))
    one_doc_as_df = pd.DataFrame.from_records(tf_idf_tuples, columns=['term', 'score']).sort_values(by='score', ascending=False).reset_index(drop=True)
    one_doc_as_df['dates'] = dates[counter]
    tfidf_results.append(one_doc_as_df)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [39]:
tfidf_df = pd.concat(tfidf_results)
tfidf_df = tfidf_df.sort_values(by=['score'], ascending=False)
tfidf_df = tfidf_df[(tfidf_df['term'] != '2002') & (tfidf_df['term'] != '2004') & (tfidf_df['term'] != '2006') & (tfidf_df['term'] != '2007') &(tfidf_df['term'] != '2008')]
tfidf_df.head(10)

,term,score,dates
0,ninch,0.750434,2000-2001
0,utorepas,0.725871,1987-1988
0,ninch,0.632659,1999-2000
1,ninch,0.588579,2001-2002
0,ninch,0.577211,1998-1999
1,ninch,0.563971,2002-2003
1,amico,0.418245,1999-2000
0,ninch,0.349532,1997-1998
0,gopher,0.347021,1993-1994
0,gopher,0.309322,1994-1995


In [33]:
a = tfidf_df[0:40].term.unique().tolist()
a_10 = a[:10]

In [34]:
top_term = tfidf_df[tfidf_df.term.isin(a_10)]

In [37]:
# Use Altair to plot this top 10 words for the entire volumn
alt.Chart(top_term).mark_bar().encode(
    x='term',
    y='score',
    color=alt.Color('term', scale=alt.Scale(scheme='accent'), sort=alt.Sort()),
)

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [72]:
# Remanent of coding
'''documents1 = humanist_vols.iloc[0:1].text.tolist()
documents2 = humanist_vols.iloc[1:2].text.tolist()
documents3 = humanist_vols.iloc[2:3].text.tolist()
documents4 = humanist_vols.iloc[3:4].text.tolist()
documents5 = humanist_vols.iloc[4:5].text.tolist()
documents6 = humanist_vols.iloc[5:6].text.tolist()
documents7 = humanist_vols.iloc[6:7].text.tolist()
documents8 = humanist_vols.iloc[7:8].text.tolist()
documents9 = humanist_vols.iloc[8:9].text.tolist()
documents10 = humanist_vols.iloc[9:10].text.tolist()
documents11 = humanist_vols.iloc[10:11].text.tolist()
documents12 = humanist_vols.iloc[11:12].text.tolist()
documents13 = humanist_vols.iloc[12:13].text.tolist()
documents14 = humanist_vols.iloc[13:14].text.tolist()
documents15 = humanist_vols.iloc[14:15].text.tolist()
documents16 = humanist_vols.iloc[15:16].text.tolist()
documents17 = humanist_vols.iloc[16:17].text.tolist()
documents18 = humanist_vols.iloc[17:18].text.tolist()
documents19 = humanist_vols.iloc[18:19].text.tolist()
documents20 = humanist_vols.iloc[19:20].text.tolist()
document_list = [documents1, documents2, documents3,documents4,documents5,documents6,documents7,documents8,documents9,documents10,documents11,documents12,documents13,documents14,documents15,documents16,documents17,documents18,documents19,documents20]'''

'documents1 = humanist_vols.iloc[0:1].text.tolist()\ndocuments2 = humanist_vols.iloc[1:2].text.tolist()\ndocuments3 = humanist_vols.iloc[2:3].text.tolist()\ndocuments4 = humanist_vols.iloc[3:4].text.tolist()\ndocuments5 = humanist_vols.iloc[4:5].text.tolist()\ndocuments6 = humanist_vols.iloc[5:6].text.tolist()\ndocuments7 = humanist_vols.iloc[6:7].text.tolist()\ndocuments8 = humanist_vols.iloc[7:8].text.tolist()\ndocuments9 = humanist_vols.iloc[8:9].text.tolist()\ndocuments10 = humanist_vols.iloc[9:10].text.tolist()\ndocuments11 = humanist_vols.iloc[10:11].text.tolist()\ndocuments12 = humanist_vols.iloc[11:12].text.tolist()\ndocuments13 = humanist_vols.iloc[12:13].text.tolist()\ndocuments14 = humanist_vols.iloc[13:14].text.tolist()\ndocuments15 = humanist_vols.iloc[14:15].text.tolist()\ndocuments16 = humanist_vols.iloc[15:16].text.tolist()\ndocuments17 = humanist_vols.iloc[16:17].text.tolist()\ndocuments18 = humanist_vols.iloc[17:18].text.tolist()\ndocuments19 = humanist_vols.iloc[18:1

In [73]:
# Remanent of coding
'''for lists in range(document_list):
    transformed_documents = vectorizer.fit_transform(lists)
    transformed_documents_as_array = transformed_documents.toarray()
    dates = humanist_vols.dates.tolist()
    tfidf_results = []
    for counter, doc in enumerate(transformed_documents_as_array):
        # construct a dataframe
        tf_idf_tuples = list(zip(vectorizer.get_feature_names(), doc))
        one_doc_as_df = pd.DataFrame.from_records(tf_idf_tuples, columns=['term', 'score']).sort_values(by='score', ascending=False).reset_index(drop=True)
        one_doc_as_df['dates'] = dates[counter]
        tfidf_results.append(one_doc_as_df)'''

"for lists in range(document_list):\n    transformed_documents = vectorizer.fit_transform(lists)\n    transformed_documents_as_array = transformed_documents.toarray()\n    dates = humanist_vols.dates.tolist()\n    tfidf_results = []\n    for counter, doc in enumerate(transformed_documents_as_array):\n        # construct a dataframe\n        tf_idf_tuples = list(zip(vectorizer.get_feature_names(), doc))\n        one_doc_as_df = pd.DataFrame.from_records(tf_idf_tuples, columns=['term', 'score']).sort_values(by='score', ascending=False).reset_index(drop=True)\n        one_doc_as_df['dates'] = dates[counter]\n        tfidf_results.append(one_doc_as_df)"

In [74]:
# Remanent of coding
'''bydate = tfidf_df[tfidf_df['dates'] == "1987-1988"].term.unique().tolist()
bydate_5 = bydate[:5]
bydate_5'''

'bydate = tfidf_df[tfidf_df[\'dates\'] == "1987-1988"].term.unique().tolist()\nbydate_5 = bydate[:5]\nbydate_5'

In [75]:
# Use Altair to plot top TFIDF terms for each volumn
alt.Chart(top_term).mark_bar().encode(
    x='dates',
    y='score',
    color=alt.Color('term', scale=alt.Scale(scheme='accent'), sort=alt.Sort()),
)

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html
